In [1]:
import operator
import math
import random
import numpy as np
from deap import algorithms, base, creator, tools, gp

In [2]:
def division_operator(numerator, denominator) : 
    if denominator==0 : 
        return 1
    return numerator/denominator

In [3]:
#평가 함수 정의
def eval_func(individual, points) : 
    #트리 표현식을 callable 함수로 변환
    func = toolbox.compile(expr=individual) 
    #MSE
    mse = ((func(x) - (2 * x**3 - 3 * x**2 - 4 * x + 1))**2 for x in points)
    
    return math.fsum(mse)/len(points)

In [11]:
#툴박스 생성 함수
def create_toolbox() : 
    pset = gp.PrimitiveSet("MAIN", 1)
    pset.addPrimitive(operator.add, 2)
    pset.addPrimitive(operator.sub, 2)
    pset.addPrimitive(operator.mul, 2)
    pset.addPrimitive(division_operator, 2)
    pset.addPrimitive(operator.neg, 1)
    pset.addPrimitive(math.cos, 1)
    pset.addPrimitive(math.sin, 1)
    
    pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
    pset.renameArguments(ARG0="x")
    
    #적합도와 개체에 대한 오브젝트 타입 정의
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)
    
    #툴박스와 register 함수 생성
    toolbox = base.Toolbox()
    toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual, toolbox.expr)
    toolbox.register("compile", gp.compile, pset=pset)
    toolbox.register("evaluate", eval_func, points=[x/10. for x in range(-10,10)])
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("mate", gp.cxOnePOint)
    toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
    
    toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height", max_value=17)))

In [20]:
random.seed(7)

toolbox = create_toolbox()
population = toolbox.population(n=450)
hall_of_fame = tools.HallOfFame(1)

Exception: Ephemerals with different functions should be named differently, even between psets.

In [16]:
stats_fit = tools.Statistics(lambda x: x.fitness.values)
stats_size = tools.Statistics(len)

In [17]:
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

In [18]:
probab_crossover = 0.4
probab_mutate = 0.2
num_generations = 60

In [19]:
population, log = algorithms.eaSimple(population, toolbox, 
                                      probab_crossover, probab_mutate, num_generations,
                                      stats=mstats, halloffame=hall_of_fame, verbose=True)

NameError: name 'population' is not defined